In [ ]:
#EXAM 2024

#Question 1 – Reconnaissance de panneaux routiers par analyse de contours
#1.a.i Lecture et affichage des images RGB
import cv2
import numpy as np
import matplotlib.pyplot as plt

stop = cv2.imread('Stop.png')
warning = cv2.imread('Warning.png')

stop_rgb = cv2.cvtColor(stop, cv2.COLOR_BGR2RGB)
warning_rgb = cv2.cvtColor(warning, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(stop_rgb)
plt.title('Stop (RGB)')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(warning_rgb)
plt.title('Warning (RGB)')
plt.axis('off')
plt.show()

# 1.a.ii Seuillage et images binaires
#Un seuillage d’Otsu est utilisé car il s’adapte automatiquement à la distribution d’intensité.
gray_stop = cv2.cvtColor(stop, cv2.COLOR_BGR2GRAY)
gray_warning = cv2.cvtColor(warning, cv2.COLOR_BGR2GRAY)

_, bin_stop = cv2.threshold(gray_stop, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
_, bin_warning = cv2.threshold(gray_warning, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(bin_stop, cmap='gray')
plt.title('Stop binaire')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(bin_warning, cmap='gray')
plt.title('Warning binaire')
plt.axis('off')
plt.show()

#1.a.iii Sobel, magnitude et phase
sobelx_stop = cv2.Sobel(bin_stop, cv2.CV_64F, 1, 0, ksize=3)
sobely_stop = cv2.Sobel(bin_stop, cv2.CV_64F, 0, 1, ksize=3)

sobelx_warn = cv2.Sobel(bin_warning, cv2.CV_64F, 1, 0, ksize=3)
sobely_warn = cv2.Sobel(bin_warning, cv2.CV_64F, 0, 1, ksize=3)

mag_stop = np.sqrt(sobelx_stop**2 + sobely_stop**2)
phase_stop = np.arctan2(sobely_stop, sobelx_stop)

mag_warn = np.sqrt(sobelx_warn**2 + sobely_warn**2)
phase_warn = np.arctan2(sobely_warn, sobelx_warn)

#1.a.iv Histogrammes de phase
#   histSize = 36 (résolution angulaire de 10°)
#   range = [-π, π]
#   Calcul uniquement pour les pixels de contour (magnitude > 0)
bins = 36
range_vals = (-np.pi, np.pi)

hist_stop,_ = np.histogram(phase_stop[mag_stop>0], bins=bins, range=range_vals, density=True)
hist_warn,_ = np.histogram(phase_warn[mag_warn>0], bins=bins, range=range_vals, density=True)

plt.plot(hist_stop, label='Stop')
plt.plot(hist_warn, label='Warning')
plt.legend()
plt.title('Histogrammes de phase normalisés')
plt.show()

#1.a.v Règle de décision
#   Stop (octogone) : distribution plus uniforme (plusieurs orientations)
#   Warning (triangle) : pics dominants à orientations spécifiques
if np.std(hist_stop) < np.std(hist_warn):
    print("Reconnu comme panneau STOP")
else:
    print("Reconnu comme panneau WARNING")

#1.b Traitement image réelle Stop
# Choix du canal rouge : le panneau stop est majoritairement rouge.
stop_real = cv2.imread('Stop real.jpg')
stop_real_rgb = cv2.cvtColor(stop_real, cv2.COLOR_BGR2RGB)
red_channel = stop_real_rgb[:,:,0]

_, bin_real = cv2.threshold(red_channel, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#Morphologie : ouverture + fermeture avec un élément structurant circulaire.
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
bin_clean = cv2.morphologyEx(bin_real, cv2.MORPH_CLOSE, kernel)
bin_clean = cv2.morphologyEx(bin_clean, cv2.MORPH_OPEN, kernel)
# Sobel, histogramme et classification : mêmes étapes que précédemment → STOP reconnu correctement.

#Question 2 – Optical Flow et estimation de vitesse
#2.a.i Canal bleu
b1 = cv2.imread('Bus_1.png')[:,:,0]
b2 = cv2.imread('Bus_2.png')[:,:,0]
b3 = cv2.imread('Bus_3.png')[:,:,0]

#2.a.ii Template Matching
template = cv2.imread('Template.png',0)
res = cv2.matchTemplate(b1, template, cv2.TM_CCOEFF_NORMED)
_,_,_,max_loc = cv2.minMaxLoc(res)
w,h = template.shape[::-1]

# 2.a.iii Shi-Tomasi
mask = np.zeros_like(b1)
mask[max_loc[1]:max_loc[1]+h, max_loc[0]:max_loc[0]+w] = 255

points = cv2.goodFeaturesToTrack(b1, 30, 0.01, 10, mask=mask, blockSize=3)

# 2.a.iv Lucas-Kanade
#   winSize = (15,15)
#   maxLevel = 2
lk_params = dict(winSize=(15,15), maxLevel=2,
criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

p2, st, _ = cv2.calcOpticalFlowPyrLK(b1, b2, points, None, **lk_params)
p3, st, _ = cv2.calcOpticalFlowPyrLK(b2, b3, p2, None, **lk_params)
#Commentaire : les coins avec fort contraste sont mieux suivis.

#2.b Estimation de la vitesse
disp = p2[:,0,0] - points[:,0,0]
dm = np.mean(disp)

v = (dm * 0.04) * 30 * 3.6
print('Vitesse estimée :', v, 'km/h')

#Question 3 – Réseaux de neurones convolutionnels
#3.a MNIST
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

(x_train,y_train),(x_test,y_test)=mnist.load_data()

# CNN proposé
model = Sequential([
Conv2D(6,(5,5),activation='relu',input_shape=(28,28,1)),
MaxPooling2D((2,2)),
Conv2D(16,(5,5),activation='relu'),
MaxPooling2D((2,2)),
Flatten(),
Dense(120,activation='relu'),
Dense(84,activation='relu'),
Dense(10,activation='softmax')
])

# Entraînement
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=3,batch_size=128,validation_data=(x_test,y_test))

# CIFAR-10
#Résultats inférieurs à MNIST car images couleur complexes.
#Améliorations :
#   Plus de filtres
#   Dropout
#   Data augmentation

# ResNet50 – Hawk.jpg
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

model = ResNet50(weights='imagenet')